<a href="https://colab.research.google.com/github/ArturHugo/PLN-2022-1/blob/main/Atividade03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 03 - Recuperação de textos
Grupo:

- Artur Hugo cunha Pereira, 180030400


In [ ]:
import re
import pandas as pd
import nltk
import numpy as np

In [ ]:
# NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Corpus Reuters

In [ ]:
nltk.download('reuters')
from nltk.corpus import reuters

cats = reuters.categories()
print("Reuters has %d categories:\n%s" % (len(cats), cats))

fileids = reuters.fileids()

categories = []
text = []

for file in fileids:
    categories.append(reuters.categories(file))
    text.append(reuters.raw(file))

df = pd.DataFrame({'ids':fileids, 'categories':categories, 'text':text})
df

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


Reuters has 90 categories:
['acq', 'alum', 'barley', 'bop', 'carcass', 'castor-oil', 'cocoa', 'coconut', 'coconut-oil', 'coffee', 'copper', 'copra-cake', 'corn', 'cotton', 'cotton-oil', 'cpi', 'cpu', 'crude', 'dfl', 'dlr', 'dmk', 'earn', 'fuel', 'gas', 'gnp', 'gold', 'grain', 'groundnut', 'groundnut-oil', 'heat', 'hog', 'housing', 'income', 'instal-debt', 'interest', 'ipi', 'iron-steel', 'jet', 'jobs', 'l-cattle', 'lead', 'lei', 'lin-oil', 'livestock', 'lumber', 'meal-feed', 'money-fx', 'money-supply', 'naphtha', 'nat-gas', 'nickel', 'nkr', 'nzdlr', 'oat', 'oilseed', 'orange', 'palladium', 'palm-oil', 'palmkernel', 'pet-chem', 'platinum', 'potato', 'propane', 'rand', 'rape-oil', 'rapeseed', 'reserves', 'retail', 'rice', 'rubber', 'rye', 'ship', 'silver', 'sorghum', 'soy-meal', 'soy-oil', 'soybean', 'strategic-metal', 'sugar', 'sun-meal', 'sun-oil', 'sunseed', 'tea', 'tin', 'trade', 'veg-oil', 'wheat', 'wpi', 'yen', 'zinc']


,ids,categories,text
0,test/14826,[trade],ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RI...
1,test/14828,[grain],CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STO...
2,test/14829,"[crude, nat-gas]",JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWA...
3,test/14832,"[corn, grain, rice, rubber, sugar, tin, trade]",THAI TRADE DEFICIT WIDENS IN FIRST QUARTER\n ...
4,test/14833,"[palm-oil, veg-oil]",INDONESIA SEES CPO PRICE RISING SHARPLY\n Ind...
...,...,...,...
10783,training/999,"[interest, money-fx]",U.K. MONEY MARKET SHORTAGE FORECAST REVISED DO...
10784,training/9992,[earn],KNIGHT-RIDDER INC &lt;KRN> SETS QUARTERLY\n Q...
10785,training/9993,[earn],TECHNITROL INC &lt;TNL> SETS QUARTERLY\n Qtly...
10786,training/9994,[earn],NATIONWIDE CELLULAR SERVICE INC &lt;NCEL> 4TH ...


# Preprocessamento dos dados

In [ ]:
# Helper functions
def tokenize(string):
  # Remove special characters
  res = re.sub(r'&lt;|\\n|>', ' ', string.lower())
  # Runs through nltk tokenizer to finish tokenization
  res = nltk.tokenize.word_tokenize(res)
  return res

def filter_stopwords(word_array, stop_list):
  # If word is in stop_list it is filtered out
  res = [word for word in word_array if word not in stop_list]
  return res

In [ ]:
texts_df = df['text']
texts_df = texts_df.sample(frac=0.5)
texts_df

6845    HEINZ &lt;HNZ> HAS HIGHER NET DESPITE HIGHER C...
3309    HANSON TRUST TO SELL U.S. CHEMICALS UNIT\n  Ha...
645     BSD BANCORP &lt;BSD> 1ST QTR NET\n  Shr seven ...
6023    &lt;FAR WEST INDUSTRIES INC> YEAR NET\n  Shr 2...
6323    WHITE ENGINE BOUGHT BY PRIVATE INVESTOR\n  Whi...
                              ...                        
4825    DU PONT &lt;DD> TO SELL CLEVELAND PLANT\n  Du ...
8691    RADA ELECTRONIC INDUSTRIES&lt;RADIF> NINE MTHS...
1176    WHITE HOUSE SAYS U.S. MONETARY POLICY CORRECT\...
4523    MORGAN &lt;JPM> PLACES BRAZIL LOANS ON NONACCR...
2845    CORBY DISTILLERIES &lt;CDL.TO> TO BUY HEUBLEIN...
Name: text, Length: 5394, dtype: object

In [ ]:
texts_df = texts_df.apply(tokenize)
texts_df

6845    [heinz, hnz, has, higher, net, despite, higher...
3309    [hanson, trust, to, sell, u.s., chemicals, uni...
645     [bsd, bancorp, bsd, 1st, qtr, net, shr, seven,...
6023    [far, west, industries, inc, year, net, shr, 2...
6323    [white, engine, bought, by, private, investor,...
                              ...                        
4825    [du, pont, dd, to, sell, cleveland, plant, du,...
8691    [rada, electronic, industries, radif, nine, mt...
1176    [white, house, says, u.s., monetary, policy, c...
4523    [morgan, jpm, places, brazil, loans, on, nonac...
2845    [corby, distilleries, cdl.to, to, buy, heublei...
Name: text, Length: 5394, dtype: object

In [ ]:
# Remove stop words
stop_list = nltk.corpus.stopwords.words('english')
stop_list.extend(['.', '\'\'', '``', '(', ')', ',', '\'s']) # Additional stopwords
texts_df = texts_df.apply(lambda text: filter_stopwords(text, stop_list))
texts_df

6845    [heinz, hnz, higher, net, despite, higher, cos...
3309    [hanson, trust, sell, u.s., chemicals, unit, h...
645     [bsd, bancorp, bsd, 1st, qtr, net, shr, seven,...
6023    [far, west, industries, inc, year, net, shr, 2...
6323    [white, engine, bought, private, investor, whi...
                              ...                        
4825    [du, pont, dd, sell, cleveland, plant, du, pon...
8691    [rada, electronic, industries, radif, nine, mt...
1176    [white, house, says, u.s., monetary, policy, c...
4523    [morgan, jpm, places, brazil, loans, nonaccrua...
2845    [corby, distilleries, cdl.to, buy, heublein, u...
Name: text, Length: 5394, dtype: object

In [ ]:
# Stemming
stemmer = nltk.stem.porter.PorterStemmer()
texts_df = texts_df.apply(lambda text: [stemmer.stem(word) for word in text])
texts_df

6845    [heinz, hnz, higher, net, despit, higher, cost...
3309    [hanson, trust, sell, u.s., chemic, unit, hans...
645     [bsd, bancorp, bsd, 1st, qtr, net, shr, seven,...
6023    [far, west, industri, inc, year, net, shr, 23,...
6323    [white, engin, bought, privat, investor, white...
                              ...                        
4825    [du, pont, dd, sell, cleveland, plant, du, pon...
8691    [rada, electron, industri, radif, nine, mth, n...
1176    [white, hous, say, u.s., monetari, polici, cor...
4523    [morgan, jpm, place, brazil, loan, nonaccru, j...
2845    [corbi, distilleri, cdl.to, buy, heublein, uni...
Name: text, Length: 5394, dtype: object

# Representações: BoW e TF-IDF

In [ ]:
# Corpus and vocabulary

corpus = []
for text in texts_df:
  for word in text:
    corpus.append(word)

vocab = set(sorted(corpus))

In [ ]:
# Counting Bag of Words

bow_count = {}
for i, text in enumerate(texts_df.values):
  bow_count[df['ids'][i]] = dict()
  for word in text:
    if word in bow_count[df['ids'][i]]:
      bow_count[df['ids'][i]][word] += 1
    else:
      bow_count[df['ids'][i]][word] = 1

bow_count_df = pd.DataFrame().from_records(bow_count).fillna(0).T.astype(int)
bow_count_df

,heinz,hnz,higher,net,despit,cost,h.j,co,said,incom,...,fitwat,872.5,cdl.to,heublin,gmhl.l,liquer,fast-expand,mcguinness',schnapp,calona
test/14826,4,1,2,2,2,1,1,1,2,1,...,0,0,0,0,0,0,0,0,0,0
test/14828,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
test/14829,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
test/14832,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
test/14833,0,0,0,0,0,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
training/1529,0,0,0,0,0,0,0,1,3,0,...,0,0,0,0,0,0,0,0,0,0
training/153,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
training/1532,0,0,0,0,0,0,0,0,4,0,...,1,0,0,0,0,0,0,0,0,0
training/1533,0,0,0,3,0,0,0,2,5,3,...,0,1,0,0,0,0,0,0,0,0


In [ ]:
# TF-IDF

document_freq = {}
for word in vocab:
  document_freq[word] = 0
  for text, word_counts in bow_count.items():
    if word in word_counts:
      document_freq[word] += 1

tf_idf  = {}
n_texts = len(texts_df.values)
for text_id, word_counts in bow_count.items():
  n_words = sum(word_counts.values())
  tf_idf[text_id] = {}
  for word, count in word_counts.items():
    tf  = count/n_words
    idf = np.log(n_texts/document_freq[word])
    tf_idf[text_id][word] = tf*idf

tf_idf_df = pd.DataFrame().from_records(tf_idf).fillna(0).T
tf_idf_df

,heinz,hnz,higher,net,despit,cost,h.j,co,said,incom,...,fitwat,872.5,cdl.to,heublin,gmhl.l,liquer,fast-expand,mcguinness',schnapp,calona
test/14826,0.245091,0.077415,0.051879,0.024458,0.068419,0.026651,0.067517,0.018578,0.008331,0.029504,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
test/14828,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011560,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
test/14829,0.000000,0.000000,0.000000,0.079848,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
test/14832,0.000000,0.000000,0.000000,0.123402,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
test/14833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027855,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
training/1529,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028641,0.019266,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
training/153,0.000000,0.000000,0.000000,0.073374,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
training/1532,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031349,0.000000,...,0.145645,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
training/1533,0.000000,0.000000,0.000000,0.027703,0.000000,0.000000,0.000000,0.028057,0.015728,0.066835,...,0.000000,0.058456,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# Recuperação de informação

In [ ]:
# Cosine similarity
def cosine_similarity(v1, v2):
  return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [ ]:
# Calcula similaride de uma notícia com todas as demais notícias
def similarity_ranking(text_id, texts_df, sim_function):
  '''
    text_id:      row's id of the text representation in the dataframe.
    texts_df:     dataframe with the texts to rank against.
    sim_function: similarity function to use in the comparisons.

    Calculate the similarity between the text in row with name text_id and 
    the remaining texts in the dataframe texts_df using sim_function.

    Returns a list o text_ids in order of most to least similar.
  '''

  # Calculate similarities
  query_text = texts_df.loc[text_id].values
  similarities_df = texts_df.apply(
      lambda text: sim_function(text.values, query_text),
      axis=1)
  
  # Remove the query text's similarity with itself
  similarities_df = similarities_df.drop(text_id)

  # Sort from most to least similar
  similarities_df = similarities_df.sort_values(ascending=False)

  # Return the similarity ranking with the id of the texts
  return similarities_df.index.to_list()

In [ ]:
# Ranking de similaridade usando BoW
bow_count_ranking = bow_count_df.apply(
    lambda text: similarity_ranking(text.name, bow_count_df, cosine_similarity),
    axis=1).to_dict()

# Ranking de similaridades usando TF-IDF
tf_idf_ranking = tf_idf_df.apply(
    lambda text: similarity_ranking(text.name, tf_idf_df, cosine_similarity),
    axis=1).to_dict()

In [ ]:
# Limitando o rankings ao top 10
bow_count_top = bow_count_ranking
for text_id in bow_count_top:
  bow_count_top[text_id] = bow_count_top[text_id][0:10]

tf_idf_top = tf_idf_ranking
for text_id in tf_idf_top:
  tf_idf_top[text_id] = tf_idf_top[text_id][0:10]

In [ ]:
# Confere se duas listas tem elementos em comum
def common(list_a, list_b):
  set_a = set(list_a)
  set_b = set(list_b)
  if set_a & set_b:
    return True
  else:
    return False

In [ ]:
# Verifica a acurácia do ranking de um dicionário de similaridade
def top_accuracies(ranking_dict, original_df):
  '''
    ranking_dict: dictionary of similarity rankings for each text in dataframe.
    original_df:  original dataframe to check categories of the rankings.

    Calculate the how many elements of the top ranking had a category in common
    with their respective query text.

    Returns a dictionary with text ids for keys and their respective accuracy
    for values.
  '''

  accuracies_dict = {}
  for text_id in bow_count_top:
    accuracies_dict[text_id] = 0
    text_categories = original_df[original_df['ids'] == text_id]['categories'].values[0]
    top_ids = bow_count_top[text_id]
    for top_id in top_ids:
      top_categories = original_df[original_df['ids'] == top_id]['categories'].values[0]
      if common(text_categories, top_categories):
        accuracies_dict[text_id] += 1
    accuracies_dict[text_id] /= len(bow_count_top[text_id])

  return accuracies_dict

In [ ]:
# Verificar acurácia dos top 10
bow_count_top_accuracies = top_accuracies(bow_count_top, df)

tf_idf_top_accuracies = top_accuracies(tf_idf_top, df)

In [ ]:
# Acurácias médias para comparar
bow_count_avg_accuracy = np.average(list(bow_count_top_accuracies.values()))
print(f'Average accuracy for BoW:    {bow_count_avg_accuracy}')
tf_idf_avg_accuracy = np.average(list(tf_idf_top_accuracies.values()))
print(f'Average accuracy for TF-IDF: {tf_idf_avg_accuracy}')